# Basic SQL Queries
(Below queries are for PostgresSQL)

This notebook provides the use case examples for basic SQL commands for querying the data (DQLcommands)

#### Schema: `Country` table

Column  | Type     |
--------|----------|
name    | varchar  |
pop     | integer  |
continent | varchar |
capital | varchar |

### 1.1. SELECT
(To extract specific columns from a table. This operation is called `Projection`)

- From country table, Select capital and population columns: <br>

		SELECT
			capital, pop
		FROM
			country;
        
- Select ALL COLUMNS (`*`) columns: <br>

		SELECT
			*
		FROM
			country;
            
- Select `DISTINCT` continent names: <br>

		SELECT
			DISTINCT continent
		FROM
			country;
		
- Select DISTINCT combination of continent and region columns. This gives unique combination of multiple column names and not unique values for any specific column: <br>

		SELECT
			DISTINCT continent, region
		FROM
			country;
		
- Select column with `AS` Alias name: <br>

		SELECT
			avg_pop AS "Average Population"
		FROM
			country;
		   

**Note:**
- SELECT is the only command in SQL that can be used for doing computations e.g. SELECT pop/100. AS avg_pop FROM country. If 100 is given without `.`, it will truncate the value to integer.
- `" "` is used for names only when identifiers have spaces between them.
- Select DISTINCT for any specific columns (Postgres specific command): <br>
    SELECT DISTINCT ON (continent) continent, region FROM country;

### CAST - AS
(Used for Datatype conversion)

		SELECT
			CAST(AVG(pop) AS integer) AS "Average Population"
		FROM
			country; 
- FLOOR() can be used for downward casting from float to integer

### 2. WHERE
(To filter rows based on condition on specific fields)

From country table, Select all capital cities in Asia: <br>

		SELECT
			country, capital
		FROM
			country
		WHERE
			continent='Asia';      
**Note:**
- Aliases cannot be used in the WHERE clause. This is because SQL first filters, then selects. So, alias is not available when WHERE clause is executed.

#### Condition Operators
- Comparison (`=`, `<>`, `<`, `<=`, `>`, `>=`)
- Pattern Matching (`LIKE`, `NOT LIKE`, `ILIKE`): ILIKE is for case insensitive
- Range (`BETWEEN`, `NOT BETWEEN`): checks if value is between two numbers, letters or times (both inclusive)
- List (`IN`, `NOT IN`): check IF value is in a list e.g. column_name [NOT] IN (val1, val2, val3)
- Null Testing (`IS NULL`)

#### Logical Operators
- `AND`, `OR` combine multiple conditions. By default, AND takes precedence over OR/ NOT. SO, we need to use `( )` to specify the order of logical operators.
- `NOT` negates a condition e.g. WHERE NOT condition

#### Pattern Matching
- Using Like, ILike
- % : any number of characters. `ESCAPE` to ignore a character: **WHERE name LIKE '%$%%' ESCAPE '$'
- _ : for single character (__ : two characters, and so on)
- `REGEXP`/ `NOT REGEXP`: performs a pattern match of a string expression against a pattern. The pattern is supplied as an argument e.g. WHERE name REGEXP '^[aeiou]', checks condition where name starts with vowel.

### 3. GROUP BY
(To group rows based on common values in a column, and extract aggregate values)

From country table, get total population of continents: <br>
    
		SELECT
			SUM(pop) AS total_population, continent
		FROM
			country
		GROUP BY
			continent;   

**Notes:**
- When GROUP BY clause is used, columns cannot be included in SELECT without aggregation functions except those columns have been used from Grouping
- Aggregation functions can be SUM, AVG, MAX, MIN, COUNT. Aggregation functions cannot be used in WHERE clause for filtering rows
- Alias can be specified (e.g. total_poulation)
- GROUP BY comes after the WHERE clause and before ORDER BY clause
- We can use multiple columns in the GROUP BY clause, and all unique combination of those columns would be returned back.

### HAVING
(Aggregation functions cannot be in WHERE clause for filtering rows. To filter using aggregate values at grouped level, HAVING clause is used)

From country table, get continents with total population > 1000000: <br>
  
		SELECT
			SUM(pop) AS total_population, continent
		FROM
			country
		GROUP BY
			continent 
		HAVING
			SUM(pop) > 1000000; 

    
Notes:
- HAVING comes just after GROUP BY, it is a part of the group by clause   
- Alias cannot be used in grouping and filtering clauses. This is because aggregate function are applied on the grouped and selected data, and alias is not available before that.
- **WHERE is used for column filters, HAVING is used for group filters**
- **WHERE is used before GROUP BY, HAVING is used after GROUP BY**

### Aggregation Functions
(Performed overall on the selected column)

- COUNT()
- MIN(), MAX()
- SUM(), AVG()

Note:
- All aggregation function ignore NULL values, except COUNT(*)
- MIN(), MAX() can be used for string values also
- Multiple aggregation functions can be used together but cannot have non-aggregated columns along with an aggregate column, unless the non-aggregated column is used in GROUP BY clause.
- Aggregation functions are executed after Selection, so, we CANNOT use aggregation functions within the WHERE clause. We use SUB-QUERIES for this.

### 4. ORDER BY
(To sort in ascending or descending order)

From country table, get total population of continents. Sort in ascending order: <br>

		SELECT
			SUM(pop) AS total_pop, continent
		FROM
			country
		GROUP BY
			continent 
		ORDER BY
			total_pop; 

**Notes:**
- Default Order is ASC (ascending), uses ORDER BY total_pop `DESC` for sorting in descending order. 
- Alias can be used in ORDER BY clause
- Can specify ORDER BY multiple columns: `ORDER BY total_pop DESC, continent ASC`. When multiple columns are used, the first is primary sort and next is secondary sort.
- ORDER BY column should appear in the SELECT clause because Ordering happens after Select.

### 5. LIMIT
(To specify the number of rows to return)

From country table, select the two largest continents in terms of total population: <br>

		SELECT
			SUM(pop) AS total_pop, continent
		FROM
			country
		GROUP BY
			continent 
		ORDER BY
			total_pop DESC
		LIMIT
			2; 
    
Note: Limit restricts the number of actual rows returned by the query, and makes the program more memory efficient. This is different from pandas dataframe .head(), where limited number of rows are visible but all are loaded in the memory.

### OFFSET
(To specify the number of rows to return, after skipping initial `offset` number of rows)

From country table, select the second and third largest continents in terms of total population (i.e skip the first row): <br>
    
		SELECT --(same as above)--
		ORDER BY
			total_pop DESC
		LIMIT
			2 OFFSET 1;    

### 6. CASE - WHEN clause

To set the value of one column conditionally depending upon the value of another column e.g.: <br>

	SELECT
		CASE
			WHEN runtime > 90 THEN 'long'
			WHEN runtime BETWEEN 30 AND 90 THEN 'normal'
			ELSE 'short'
		END AS run_type
        
General format for CASE-WHEN is:

	SELECT
		CASE
			WHEN condition_1 THEN expression_1
			WHEN condition_2 THEN expression_2
			ELSE expression_3
		END AS alias

### 7. JOINS
(To retrieve data from two or more tables simultaneously and create virtual tables from a database)

		SELECT
			columns
		FROM
			left_table
			JOIN_TYPE
			right_table
			ON
			join_condition
		WHERE
			row_filter;

**Type of Joins**

**A. CROSS_JOIN**
- Combines all rows of one table with all rows of the other table, without any condition (m x n rows)

		SELECT
			t1.col1, t2.col2
		FROM
			left_table t1
			CROSS JOIN
			right_table t2
			ON
			join_condition;

Note: 
- If there are common column names, we use simple table aliases or full table name before the column names.
- Table aliases are valid only for the duration of the query
- Each join is initially a CROSS JOIN, and then filtered based on condition.
- If alias is given, then we have to use the alias all the time, throughout the query, because we have decided to change the name of the table.
- `USING` can be used instead of `ON`, to select the columns whose value needs to be matched </br>
   SELECT * FROM table1 AS t1 JOIN table2 AS t2 USING (col1, col2, col3)

**B. INNER_JOIN**
- Intersection: only retrieves rows that are common to both the tables
- Only returns the records for which key are matched between the two or more tables

		SELECT
			t1.col1, t2.col2
		FROM
			left_table t1
			INNER JOIN
			right_table t2
			ON
			t1.col3 = t2.col3;

**C. SELF_JOIN**
- To Join all rows from one table with itself.
- Use case: Find pairs of countries that are in the same continent
- Same table has two aliases

		SELECT
			t1.name, t2.name
		FROM
			country t1
			JOIN
			country t2
			ON
			t1.continent = t2.continent
			AND t1.id <> t2.id;

Note: t1.id <> t2.id is used to remove self mapped names

**D. NATURAL JOIN**
- Automatically joins all columns with similar values

**E. LEFT OUTER JOIN / LEFT JOIN**
- All values from left table are returned and only matching values from the right table are returned. If no matching values are found in the right table, NULL is returned.

**F. RIGHT OUTER JOIN / RIGHT JOIN**
- All values from right table are returned and only matching values from the left table are returned. If no matching values are found in the left table, NULL is returned.

**G. FULL OUTER JOIN / FULL JOIN**
- Full join combines the left Outer Join and Right Outer Join
- Cross Join matches all rows regardless of any condition and has no NULL Values. Full join first finds the matching rows and then adds the non-matching rows with NULL values.

### 8. SUBQUERIES

A SELECT statement that can be used inside another SQL statement. So, subquery is a nested/ INNER query within a larger/OUTER query.

Notes:
- SUB-QUERIES must be enclosed in `( )` and should not end with `,`
- Can be used in SELECT, FROM, WHERE, HAVING or other clauses
- SUBQUERIES vs JOIN: Subqueries are generally more readable but they do not give access to columns in the INNER table. Joins are generally faster but joins give a lot of redundant data during join and then filter, so it is more memory intensive. 

**SIMPLE/ UNCORRELATED SUBQUERY:** A subquery where the inner query is completely independent of the outer query

e.g. SELECT countries whose area is more than the average area for all countries

		SELECT
			name
		FROM
			country
		WHERE
			area > 
			(SELECT AVG(area) FROM country);

**CORRELATED SUBQUERY:** The outer query is dependent upon what is happening in the inner query. In this SQL goes row by row in the original query and compares it to each row in the inner query i.e. every row in a table is compared against related data. Correlated subquery is used when the subquery has to return different results for each row of the original query. This is analogous to nested for-loop in python. It has big-O of n^2. Hence, it is very slow.

e.g. SELECT country from each continent which has the maximum population in that continent

		SELECT
			c1.name, c1.continent
		FROM
			country c1
		WHERE
			c1.population = 
			(SELECT max(c2.population) FROM country c2 
			WHERE c2.continent = c1.continent);

#### ANY / ALL

These key words are used to check if the values in outer query are equal to `Any` or `All` values in the inner query.

#### EXISTS / NOT EXISTS

This key word is used to determine if the sub-query returns any row. It can be used in simple or correlated queries but are typically used in correlated sub-queries.

#### UNION, INTERSECT, EXCEPT 

(To combine output from two or more queries)

		(Query-1) 
		UNION/ INTERSECT/ EXCEPT
		(Query-2)

### 9. Windows Functions

A window is a set of rows defined using the `OVER` clause. The Window functions are used to apply aggregation and ranking functions over a particular window. 

Various aggregate functions such as SUM(), COUNT(), AVERAGE(), MAX(), and MIN() applied over a particular window (set of rows) are called aggregate window functions.

#### OVER

- Partitions rows to form set of rows: `OVER(PARTITION BY condition)`
- Orders rows within the partitions in a particular order (`ORDER BY`)

#### Aggregation OVER Partitions

e.g. get average population for each continent

		SELECT
			name, pop, continent,
			ROUND(AVG(pop) OVER(PARTITION BY continent)) AS avg_pop_by_cont
		FROM
			country
		ORDER BY
			pop DESC;

#### Ranking OVER Partitions
Rank() function assigns a rank to each row within a partition of a result set.

e.g. order by the population (descending). A rank value of 1 represents the highest population.

		SELECT
			name, pop,
			RANK() OVER(ORDER BY pop DESC)
		FROM
			country

### 10. Basic Operators on DATA TYPES

### Arithematic

- Operators: +, -, *, /, % (modulo, remainder), ^ (exponent), @ (absolute value, without sign)
- Functions: abs(), ceil(), exp(), floor(), ln(), log(), pi(), power(), round(), sqrt(), round() works only with NUMERIC data types and not with REAL

### String

- Concatenation `||`. Non string values are type casted to String.
- length(), lower(), upper(), position(substring in string): count starts with 1, 
- substring(string, from position, for characters), right(string, number of characters), left(string, number of characters)

### Datetime

- +, -, *, / : works on datetime objects
- CURRENT_DATE, NOW(), CURRENT_TIME(0), CURRENT_TIMESTAMP(0), LOCALTIME(0), LOCALTIMESTAMP(0) : Number represents precision of the date-time object
- `EXTRACT` is used to take out bits of date-time object: EXTRACT (hour FROM NOW())
- age() function to calculate age from start
- PostgreSQL specific functions to extract values from date columns: </br>
SELECT </br>
date_part('year',date_col) as year, </br>
date_part('month',date_col) as month, </br>
date_part('day',date_col) as day, </br>
date_part('dow',date_col) as dayofweek, </br>
to_char(date_col, 'Dy') as day_name, </br>
to_char(date_col,'Month') as month_name, </br>
date_col </br>
FROM table_name

#### Order of Operations: Arithmetic (+, *) >> Comparison (<, >=) >> Logical (AND, OR)

### 11. Miscellaneous Commands

#### Timezone

- Show current timezone: SHOW TIMEZONE
- Change current timezone: SET timezone = 'America/New_York'

#### NULLIF
(Returns NULL value if column contains a certain value)	
	
	SELECT
		NULLIF(column_name, value)
	FROM
		table_name

#### Comments

- `--` for single line or inline comments
- `\*     *\` for multi-line comments 

#### Version
- To show version of Postgres: SELECT version()